In [ ]:
# List of strings of LAST FOUR DIGITS of tag EPCs to be used in calculations.
TAGS = [
    # '2B42', '3943', '3242',
    '3734', '3842', '0B5D'
]

# Name of the folder where outputted files will be placed.
OUTPUT_NAME = 'Collation Results'

# Name of the folder where
DATA_DIREC = 'Collation of Constant T Data'

In [ ]:
%%capture

import numpy as np
import matplotlib.pyplot as plt
import csv, datetime, os
import matplotlib.dates as mdates
import pandas as pd
from sklearn.metrics import r2_score
import pandas as pd


# HELPER FUNCTIONS FOR LOADING DATA
# These should not cause errors--if they do, let me know.

def MakeDataFromText(filepath:str):
    """
    Helper function to parse text files in IFM proprietary format.
    """
    with open(filepath, 'r') as file:
        reader = csv.reader(file)

        db = dict()                          # Dictionary that will hold k,v pairs of 
        recent_EPC = ''                      # The most recent EPC that was read, and data is being assigned to
        Time_Or_Data = True                  # Time is true, Data is false
        reached_24 = False

        for l in reader:                     # Iterate over entire text file
            line = l[0]                      # Each line only has one token, so we select it
            
            if len(line)==24:                # If it is an EPC
                db[line] = [[],[]]           # Create new k,v pair in database
                recent_EPC = line            # Current EPC to add to is selected
            
            elif line == '[':                # If start bracket, beginning to collect an array of data
                data = []

            elif line == ']':                # End bracket means end of data array
                if len(data)!=0:
                    if Time_Or_Data:
                        db[recent_EPC][0] = data
                        Time_Or_Data = False
                    else:
                        db[recent_EPC][1] = data
                        Time_Or_Data = True
                data = []

            elif line.count(":")==2:
                h,m,s = line.split(':')      # Parse timestamp into hours, minutes, seconds

                if int(h)==23:
                    reached_24 = True        # If end of day reached, must move to Day 2

                if reached_24 and int(h) < 12:
                    day=2
                else:
                    day=1

                data.append(datetime.datetime(1970, 2, day, int(h), int(m), int(s)))
            
            elif line ==' ':               # If blank space, then we skip to parse next line
                pass

            else:                          # If it is a data point
                try:
                    data.append(float(line))
                except:
                    pass                   # Error catching case just in case, so annoying errors don't crash program
    return db

def MakeDict(data:list):
    """
    """
    t1234 = dict()
    for d in data:
        for k,v in d.items():
            if k in t1234: # if tag is already in dict
                for i in range(len(v[0])):
                    if v[0][i] in t1234[k]:
                        t1234[k][v[0][i]].append(v[1][i])
                    else:
                        t1234[k][v[0][i]] = [v[1][i]]
            else:             # if tag is not in dict
                t1234[k] = dict()
                for i in range(len(v[0])):
                    t1234[k][v[0][i]] = [v[1][i]]
    for k,v in t1234.items():
        for k2,v2 in v.items():
            t1234[k][k2] = sum(v2)/len(v2)
    return t1234


############################################################################################################################
# DO NOT CHANGE CODE IN THIS SECTION, FOLLOW ERROR MESSAGE #################################################################
############################################################################################################################

try:    os.mkdir(OUTPUT_NAME) # Create output folder if it does not exist
except: pass

try:    os.mkdir(os.path.join(OUTPUT_NAME, 'Plots'))
except: pass

DATA_NAMES, TEMP_LIST, RSSI_LIST = [], [], []
for D__ in os.listdir(DATA_DIREC):
    DATA_NAMES.append(D__)         # List of all data file names in DATA_DIREC

    path = os.path.join(DATA_DIREC, D__)
    t,r = dict(), dict()
    for prefix in os.listdir(path):
        filepath = os.path.join(path, prefix)
        if   prefix[:4]=='tags':
            TEMP_LIST.append( MakeDict( [MakeDataFromText(filepath)] ) ) # Create a temperature dictionary (hash table)
        elif prefix[:4]=='RSSI':
            RSSI_LIST.append( MakeDict( [MakeDataFromText(filepath)] ) ) # Create an RSSI dictionary (hash table)

    # print(len(TEMP_LIST), len(RSSI_LIST))
    assert len(TEMP_LIST)==len(RSSI_LIST)==len(DATA_NAMES), "Format Error: {} does not contain a tags_NUM.csv and RSSI_NUM.csv".format(path)

TEMP_LIST, RSSI_LIST, DATA_NAMES = np.array(TEMP_LIST), np.array(RSSI_LIST), np.array(DATA_NAMES) # NP arrays for speed
############################################################################################################################

In [ ]:
%%capture

RSSI_LABEL = [ 'All RSSIs', 'RSSI less than or equal to 20', 'RSSI greater than 20' ]

for a in range(len(DATA_NAMES)):  # Iterate over all data files
    for RSSI_VAL in range(3):     # 0 = All RSSI, 1 = RSSI<=20, 2 = RSSI>20

        fig, axs = plt.subplots(len(TAGS), 1, figsize=(14, len(TAGS)*4), sharex=True)

        for b in range(len(TAGS)):
            axs[b].set_title(TAGS[b] + ", " + DATA_NAMES[a] + ", " + RSSI_LABEL[RSSI_VAL])
            axs[b].set_xlabel("RSSI (dBm)")
            axs[b].set_ylabel("Temperature (C)")

            for k,v in TEMP_LIST[a].items():
                if k[-4:]==TAGS[b]:
                    st, sr = sorted( TEMP_LIST[a][k].items() ), sorted( RSSI_LIST[a][k].items() )
                    dt, tt, dr = [ j[1] for j in st ], [ j[0] for j in st ], [ j[1] for j in sr ]

                    # Making Standard Deviation and Error Bars
                    means, rssi = [], []
                    c1 = { i:[] for i in range(27) }
                    for i in range(len(dt)): 
                        c1[int(dr[i])].append(dt[i])
                    
                    for k,v in c1.items():
                        if   RSSI_VAL==0 and len(v)>2:
                            mean = sum(v)/len(v); std = np.std(v); rssi.append(k); means.append(mean)
                            axs[b].scatter(k, mean, marker='o',color='b')
                            axs[b].errorbar(k, mean, yerr=std, color='b')
                            axs[b].text(
                                k, mean+std, str(round(std, 3)), fontsize=10, color="b", horizontalalignment='center', verticalalignment='bottom'
                            )
                        elif RSSI_VAL==1 and len(v)>2 and RSSI_VAL<=20:
                            mean = sum(v)/len(v); std = np.std(v); rssi.append(k); means.append(mean)
                            axs[b].scatter(k, mean, marker='o',color='b')
                            axs[b].errorbar(k, mean, yerr=std, color='b')
                            axs[b].text(
                                k, mean+std, str(round(std, 3)), fontsize=10, color="b", horizontalalignment='center', verticalalignment='bottom'
                            )
                        elif RSSI_VAL==2 and len(v)>2 and RSSI_VAL>20:
                            mean = sum(v)/len(v); std = np.std(v); rssi.append(k); means.append(mean)
                            axs[b].scatter(k, mean, marker='o',color='b')
                            axs[b].errorbar(k, mean, yerr=std, color='b')
                            axs[b].text(
                                k, mean+std, str(round(std, 3)), fontsize=10, color="b", horizontalalignment='center', verticalalignment='bottom'
                            )

                    if RSSI_VAL==0: axs[b].set_xticks(range(27))
                    if RSSI_VAL==1: axs[b].set_xticks(range(21))
                    if RSSI_VAL==2: axs[b].set_xticks(range(21,27))

                    axs[b].xaxis.set_visible(True)
                    
                    # axs[b].set_title(TAGS[b] + ", " + RSSI_LABEL[RSSI_VAL])
                    # axs[b].set_xlabel("RSSI (dBm)")
                    # axs[b].set_ylabel("Temperature (C)")

                    fig.subplots_adjust(hspace=0.4)
                            
                    # fig.savefig(OUTPUT_NAME + "/Plots/" + DATA_NAMES[a] + ", " + TAGS[b] + ", " + RSSI_LABEL[RSSI_VAL] + ".png")
                    fig.savefig(OUTPUT_NAME + "/Plots/" + DATA_NAMES[a] + ", " + RSSI_LABEL[RSSI_VAL] + ".png")
                    # fig.clf()
                    # fig.clear()
                    # plt.close()